## structure
- get page
    - get link
        - get article
            - get content
            - get image
            - OCR

## data format
- type wrong | partial wrong | correct
- title
- content
- category
- time
- report
- link


## collect

### connect drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### update teseract v5 ppa

In [2]:
# !rm /opt/conda/lib/libcurl.so.4
# !ln -s /usr/lib/x86_64-linux-gnu/libcurl.so.4.8.0 /opt/conda/lib/libcurl.so.4
!add-apt-repository ppa:alex-p/tesseract-ocr5 -y
!apt update
!apt install -y tesseract-ocr
!apt install tesseract-ocr-eng
# If you want all languages -- please note this will take time to download. You could also point to the already install tessdata in 4.00 as well.
# !apt install tesseract-ocr-all -y
# !tesseract --version

Repository: 'deb https://ppa.launchpadcontent.net/alex-p/tesseract-ocr5/ubuntu/ jammy main'
Description:
 The Tesseract OCR engine was one of the top 3 engines in the 1995
 UNLV Accuracy test. Between 1995 and 2006 it had little work done on
 it, but since then it has been improved extensively by Google and is
 probably one of the most accurate open source OCR engines
 available. It can read a wide variety of image formats and convert
 them to text in over 40 languages. This package includes the command
 line tool.
More info: https://launchpad.net/~alex-p/+archive/ubuntu/tesseract-ocr5
Adding repository.
Found existing deb entry in /etc/apt/sources.list.d/alex-p-ubuntu-tesseract-ocr5-jammy.list
Adding deb entry to /etc/apt/sources.list.d/alex-p-ubuntu-tesseract-ocr5-jammy.list
Found existing deb-src entry in /etc/apt/sources.list.d/alex-p-ubuntu-tesseract-ocr5-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/alex-p-ubuntu-tesseract-ocr5-jammy.list
Adding key to /etc/

### install teseract-ocr v5

In [3]:
!sudo apt install tesseract-ocr-chi-tra
!sudo apt install tesseract-ocr-chi-sim
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-chi-tra is already the newest version (1:5.0.0~git39-6572757-2ppa1~jammy1).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-chi-sim is already the newest version (1:5.0.0~git39-6572757-2ppa1~jammy1).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.


In [4]:
!tesseract --version | cat
!tesseract --list-langs | cat

tesseract 5.3.4
 leptonica-1.82.0
  libgif 5.1.9 : libjpeg 8d (libjpeg-turbo 2.1.1) : libpng 1.6.37 : libtiff 4.3.0 : zlib 1.2.11 : libwebp 1.2.2 : libopenjp2 2.4.0
 Found AVX2
 Found AVX
 Found FMA
 Found SSE4.1
 Found OpenMP 201511
 Found libarchive 3.6.0 zlib/1.2.11 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.4.8
 Found libcurl/7.81.0 OpenSSL/3.0.2 zlib/1.2.11 brotli/1.0.9 zstd/1.4.8 libidn2/2.3.2 libpsl/0.21.0 (+libidn2/2.3.2) libssh/0.9.6/openssl/zlib nghttp2/1.43.0 librtmp/2.3 OpenLDAP/2.5.16
List of available languages in "/usr/share/tesseract-ocr/5/tessdata/" (4):
chi_sim
chi_tra
eng
osd


### import

In [5]:
import pytesseract
import os,re,random
import pandas as pd
try:
    from PIL import Image
except ImportError:
    import Image
from bs4 import BeautifulSoup
import requests

### get website with UA

In [6]:
def get_html(url):
    # headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36"}
    headers = ['Mozilla/5.0 (Windows NT 6.1; WOW64; rv:41.0) Gecko/20100101 Firefox/41.0',
               'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36',
               'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
               'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.71 Safari/537.36',
               'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11) AppleWebKit/601.1.56 (KHTML, like Gecko) Version/9.0 Safari/601.1.56',
               'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
               'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_1) AppleWebKit/601.2.7 (KHTML, like Gecko) Version/9.0.1 Safari/601.2.7',
               'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
               'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:41.0) Gecko/20100101 Firefox/41.0',
               'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:41.0) Gecko/20100101 Firefox/41.0',
               'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:41.0) Gecko/20100101 Firefox/41.0',
               'Mozilla/5.0 (Windows NT 6.3; WOW64; rv:41.0) Gecko/20100101 Firefox/41.0',
               'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:41.0) Gecko/20100101 Firefox/41.0',
               'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36',
               'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
               'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.71 Safari/537.36',
               'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
               'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36',
               'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
               'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
               'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
               'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
               'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:51.0) Gecko/20100101 Firefox/51.0',
               ]
    headers = {'User-Agent': headers[random.randrange(0, len(headers))]}
    res = requests.get(url,headers=headers)
    return res


### get all linlks

In [7]:
def get_pages():
    url = f"{site_url}/articles/report?page="
    done = False
    i = 0
    links = []
    while not done:
        res = get_html(f"{url}{i}")
        res.encoding = 'utf-8'
        if res.status_code != 200:
            done = True
        else:
            print(i,len(links))
            tmp,done =  get_links(res.text)
            links.extend(tmp)
        i += 1
    return links

def get_links(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    pattern = re.compile(r'^/articles/\d+')
    page = soup.select(".moscone-flipped-content")
    # print(len(page),end=" ")
    links = page[0].find_all("a",href=pattern)
    links = [i.get('href') for i in links]
    links = list(set(links))
    # print(len(links))
    # print(links)
    if len(links) == 0:
        return [],True
    else:
        return links,False

### get images

In [8]:
def get_image(soup,article_id,path):
    page = soup.select(".content-inner")[0]
    imgs = page.find_all('img')
    src = [i['src'] for i in imgs]
    idx = 0
    images = []
    img_links = []
    path = f"{path}/images"
    for i in src:
        link = f"{site_url}{i}"
        try:
            res = get_html(link)
        except Exception as e:
            continue
            print(link)
            print(e)
        file_name = f"{article_id}_{idx}{os.path.splitext(i)[1]}"
        file_path = os.path.join(path, file_name)
        images.append(file_path)
        img_links.append(link)
        # print(file_path)
        if res.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(res.content)
        idx += 1
    return images,img_links

### ocr

In [9]:
def ocr(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image, lang='chi_tra')
    # print(text)
    return text

### get article

In [10]:
def get_article(url_id,path):
    url = f"{site_url}{url_id}"
    match_map = re.search(r'articles/',url_id)
    article_id = url_id[match_map.end():]
    is_article = True
    article = {}

    article["link"] = url
    article["article_id"] = article_id

    res = get_html(url)
    res.encoding = 'utf-8'
    html_doc = res.text
    soup = BeautifulSoup(html_doc, 'html.parser')

    title = soup.select(".entity-list-title")[0].text
    reply = soup.select(".node-preface")[0].text
    header = soup.select(".content-inner")[0].select(".node-header")[0]
    time = header.select(".submitted")[0].text
    tags = header.select(".node-tags")[0].find_all("a")
    tags = [i.text for i in tags]
    rumor_type = tags[1]
    category = tags[2]
    imgs,img_links = get_image(soup,article_id,path)
    # print("img error",e)

    if title.find("影片") != -1 or title.find("圖片") != -1 or title.find("照片") != -1:
        is_article = False
        return is_article,article
    idx = 0
    for i in range(len(imgs)):
        tmp = ocr(imgs[i])
        if tmp:
            article[f"ocr_{idx}"] = tmp
            article[f"img_{idx}"] = img_links[i]
            idx += 1
    article["rumor_type"] = rumor_type
    article["category"] = category
    article["time"] = time
    article["title"] = title
    article["reply"] = reply
    return is_article,article

### main

In [11]:
site_url = 'https://tfc-taiwan.org.tw'
path = "/content/drive/MyDrive/ML/project/graduation/專題資料/"
pages = get_pages()
print(len(pages))

0 0
1 10
2 20
3 30
4 40
5 50
6 60
7 70
8 80
9 90
10 100
11 110
12 120
13 130
14 140
15 150
16 160
17 170
18 180
19 190
20 200
21 210
22 220
23 230
24 240
25 250
26 260
27 270
28 280
29 290
30 300
31 310
32 320
33 330
34 340
35 350
36 360
37 370
38 380
39 390
40 400
41 410
42 420
43 430
44 440
45 450
46 460
47 470
48 480
49 490
50 500
51 510
52 520
53 530
54 540
55 550
56 560
57 570
58 580
59 590
60 600
61 610
62 620
63 630
64 640
65 650
66 660
67 670
68 680
69 690
70 700
71 710
72 720
73 730
74 740
75 750
76 760
77 770
78 780
79 790
80 800
81 810
82 820
83 830
84 840
85 850
86 860
87 870
88 880
89 890
90 900
91 910
92 920
93 930
94 940
95 950
96 960
97 970
98 980
99 990
100 1000
101 1010
102 1020
103 1030
104 1040
105 1050
106 1060
107 1070
108 1080
109 1090
110 1100
111 1110
112 1120
113 1130
114 1140
115 1150
116 1160
117 1170
118 1180
119 1190
120 1200
121 1210
122 1220
123 1230
124 1240
125 1250
126 1260
127 1270
128 1280
129 1290
130 1300
131 1310
132 1320
133 1330
134 1340
135 13

KeyboardInterrupt: 

In [ ]:
data = []
not_article = []
j = 0
for i in pages:
    print(i,j)
    is_article,article = get_article(i,path)
    if is_article:
        data.append(article)
    else:
        not_article.append(article)
    j += 1
print(len(data),len(not_article))

### test

In [ ]:
len(data)

In [ ]:
# # test
# site_url = 'https://tfc-taiwan.org.tw'
# path = "/content/drive/MyDrive/ML/project/graduation/"
# url = "/articles/10473"
# t,article = get_article(url,path)
# print(article.keys())


### save to csv

In [ ]:
not_article

In [ ]:
df = pd.DataFrame(data)
df2 = pd.DataFrame(not_article)
display(df)
df.to_csv(os.path.join(path,"test_data_original.csv"))
df2.to_csv(os.path.join(path,"not_article.csv"))

## post process